### Preamble

#### Project Template

In [ ]:
%load_ext autoreload

In [ ]:
import os as _os
_os.chdir(_os.environ['PROJECT_ROOT'])
_os.path.realpath(_os.path.curdir)

#### Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
from lib.pandas_util import idxwhere, aligned_index, align_indexes, invert_mapping
import matplotlib as mpl
import lib.plot
import statsmodels as sm
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm
import subprocess
from tempfile import mkstemp
import time
import subprocess
from itertools import chain
import os
from itertools import product
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
import sfacts as sf

In [ ]:
import lib.thisproject.data

#### Set Style

In [ ]:
sns.set_context('talk')
plt.rcParams['figure.dpi'] = 50

## Set Parameters / Load Data

In [ ]:
Fixed params

group='xjin_hmp2'
stemA='r.proc'
centroid=75
stemB = 'filt-poly05-cvrg05.ss-g10000-block0-seed0.fit-sfacts37-s85-seed0'
gene_params = f"99-v22-agg{centroid}"
depth_thresh="250"
specgene_params='specgene-ref-t25-p95'

In [ ]:
ref_strains = pd.read_table('meta/genome.tsv', index_col='genome_id')[lambda x: ~x.genome_path.isna()]
species_list = ref_strains.species_id.unique()

In [ ]:
# TODO: Make these loop variables
seed = 0
max_strain_samples = 1

xjin_benchmarking = []
for max_strain_samples in [-1, 1]:
    if max_strain_samples > 0:
        ss_params = f"seed{seed}-n{max_strain_samples}"
    else:
        ss_params = "all"
    for corr_thresh in [0, 0.35]:
        corr_thresh_key = int(corr_thresh * 1000)
        for species in species_list:
            path = f"data/group/{group}/species/sp-{species}/{stemA}.gtpro.{stemB}.gene{gene_params}.spgc_{specgene_params}_ss-xjin-{ss_params}_thresh-corr{corr_thresh_key}-depth{depth_thresh}.xjin_strain_summary.tsv"
            if not os.path.exists(path):
                print(path)
                continue

            xjin_benchmarking.append(pd.read_table(path).assign(
                species=species,
                seed=seed,
                max_strain_samples=max_strain_samples,
                corr_thresh=corr_thresh,
            ))

xjin_benchmarking = pd.concat(xjin_benchmarking).assign(
    # Reasonable filters:
    to_drop=lambda x: ( False
        | (x.total_num_reference_genomes > 1)
        | (x.strain != x.dominant_strain)
        | ((x.num_strain_samples != x.max_strain_samples) & (x.max_strain_samples > 0))
    )
)

xjin_benchmarking

In [ ]:
d = (
    xjin_benchmarking
    [~xjin_benchmarking.to_drop]
    .set_index(['species', 'genome_id', 'corr_thresh', 'max_strain_samples'])
    .f1
    .unstack(['max_strain_samples', 'corr_thresh'])
    .rename(columns={0.00: 'depth_only', 0.35: 'corr_filt'})
    .rename(columns={-1: 'all_samples', 1: 'one_sample'})
)

plt.scatter('depth_only', 'corr_filt', data=d['all_samples'], label='all_samples')
plt.scatter('depth_only', 'corr_filt', data=d['one_sample'], label='one_sample')
plt.plot([0, 1], [0, 1])
plt.legend()